在运行dw相关表之前，首先运行下面部分以创建中间表

In [1]:
import pandas as pd
from pyspark import SparkContext,SQLContext
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.types import StringType

In [2]:
spark=SparkSession \
        .builder \
        .appName('renhang_etl') \
        .master('yarn')\
        .getOrCreate()

In [3]:
#创建DW.dw_baseinfo_detail
table1 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "hyxf.ICRMESSAGEHEADER") \
    .option("user", "etl") \
    .option("password", "%TGRDJh3Hg2e4f") \
    .load()
table1.createOrReplaceTempView("ICRMESSAGEHEADER")

table2 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "hyxf.ICRCREDITCUE") \
    .option("user", "etl") \
    .option("password", "%TGRDJh3Hg2e4f") \
    .load()
table2.createOrReplaceTempView("ICRCREDITCUE")

table3 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "hyxf.ICRLOANCARDINFO") \
    .option("user", "etl") \
    .option("password", "%TGRDJh3Hg2e4f") \
    .load()
table3.createOrReplaceTempView("ICRLOANCARDINFO")

table4 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "hyxf.ICRCREDITCUE") \
    .option("user", "etl") \
    .option("password", "%TGRDJh3Hg2e4f") \
    .load()
table4.createOrReplaceTempView("ICRCREDITCUE")

table5 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "hyxf.ICRLATEST2YEAROVERDUECARD") \
    .option("user", "etl") \
    .option("password", "%TGRDJh3Hg2e4f") \
    .load()
table5.createOrReplaceTempView("ICRLATEST2YEAROVERDUECARD")

table6 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "hyxf.ICRLATEST5YEAROVERDUEDETAIL") \
    .option("user", "etl") \
    .option("password", "%TGRDJh3Hg2e4f") \
    .load()
table6.createOrReplaceTempView("ICRLATEST5YEAROVERDUEDETAIL")

table7 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "hyxf.ICRLATEST2YEAROVERDUE") \
    .option("user", "etl") \
    .option("password", "%TGRDJh3Hg2e4f") \
    .load()
table7.createOrReplaceTempView("ICRLATEST2YEAROVERDUE")

table8 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "hyxf.ICRLOANINFO") \
    .option("user", "etl") \
    .option("password", "%TGRDJh3Hg2e4f") \
    .load()
table8.createOrReplaceTempView("ICRLOANINFO")

In [4]:
spark.sql("""
select 
		ordue.reportno,
		ordue.ACCOUNT,
		head.QUERYTIME,
		ordue.MONTH,
		month(date_format(head.QUERYTIME,'%Y-%m-%d')) - month(date_format(concat(ordue.month,'.01'),'%Y-%m-%d')) as gap_months,
		case 
			when ordue.LASTMONTHS in ('G','Z','D') then 8 
			else ordue.LASTMONTHS 
		end as LASTMONTHS ,
		loanfo.GUARANTEETYPE,STATE,FINANCEORG
from ICRLATEST2YEAROVERDUECARD ordue
-- 只取贷记卡的信息
inner join ICRLOANCARDINFO loanfo on ordue.reportno=loanfo.reportno and ordue.ACCOUNT=loanfo.ACCOUNT 
left join ICRMESSAGEHEADER head on ordue.reportno=head.reportno
where BIZTYPE='准贷记卡' and ordue.LASTMONTHS!='C'
""").createOrReplaceTempView("temp_standcard_2yearoverdue")


spark.sql("""
select 
		ordue.reportno,
		ordue.ACCOUNT,
		head.QUERYTIME,
		ordue.MONTH,
		month(date_format(head.QUERYTIME,'%Y-%m-%d')) - month(date_format(concat(ordue.month,'.01'),'%Y-%m-%d')) as gap_months,
		case 
			when ordue.LASTMONTHS in ('G','Z','D') then 8 
			else ordue.LASTMONTHS 
		end as LASTMONTHS ,
		loanfo.GUARANTEETYPE,STATE,FINANCEORG
from ICRLATEST2YEAROVERDUECARD ordue
-- 只取贷记卡的信息
inner join ICRLOANCARDINFO loanfo on ordue.reportno=loanfo.reportno and ordue.ACCOUNT=loanfo.ACCOUNT 
left join ICRMESSAGEHEADER head on ordue.reportno=head.reportno
where BIZTYPE='贷记卡' and ordue.LASTMONTHS!='C'
""").createOrReplaceTempView("temp_creditcard_2yearoverdue")


spark.sql("""
select 
		ordue.reportno,
		ordue.ACCOUNT,
		head.QUERYTIME,
		ordue.MONTH,
		month(date_format(head.QUERYTIME,'%Y-%m-%d')) - month(date_format(concat(ordue.month,'.01'),'%Y-%m-%d')) as gap_months,
		ordue.LASTMONTHS,ordue.AMOUNT,loanfo.GUARANTEETYPE,STATE,FINANCEORG
from ICRLATEST5YEAROVERDUEDETAIL ordue
-- 只取贷记卡的信息
inner join ICRLOANCARDINFO loanfo on ordue.reportno=loanfo.reportno and ordue.ACCOUNT=loanfo.ACCOUNT 
left join ICRMESSAGEHEADER head on ordue.reportno=head.reportno
where month!='--'  and BIZTYPE='贷记卡'
""").createOrReplaceTempView("temp_creditcard_overdue")


spark.sql("""
select 
		reportno,
		ACCOUNT,
		QUERYTIME,
		MONTH,
		gap_months,
		LASTMONTHS,
		GUARANTEETYPE,
		STATE,
		FINANCEORG 
from temp_creditcard_2yearoverdue
union 
select 
		reportno,
		ACCOUNT,
		QUERYTIME,
		date_format(concat(month,'.01'),'%Y-%m'),
        gap_months,
		LASTMONTHS,
		GUARANTEETYPE,
		STATE,
		FINANCEORG 
from temp_creditcard_overdue
""").createOrReplaceTempView("temp_creditcard_5yearoverdue")



spark.sql("""
select 
		ordue.reportno,
		ordue.ACCOUNT,
		head.QUERYTIME,
		ordue.MONTH,
		month(date_format(head.QUERYTIME,'%Y-%m-%d')) - month(date_format(concat(ordue.month,'.01'),'%Y-%m-%d')) as gap_months,
		case 
			when ordue.LASTMONTHS in ('G','Z','D') then 8 
			else ordue.LASTMONTHS 
		end as LASTMONTHS ,
		loanfo.GUARANTEETYPE,
		STATE,
		FINANCEORG
from ICRLATEST2YEAROVERDUE ordue
-- 只取贷款的信息
inner join ICRLOANINFO loanfo on ordue.reportno=loanfo.reportno and ordue.ACCOUNT=loanfo.ACCOUNT 
left join ICRMESSAGEHEADER head on ordue.reportno=head.reportno
where ordue.LASTMONTHS!='C'
""").createOrReplaceTempView("temp_loan_2yearoverdue")



spark.sql("""
select 
		ordue.reportno,
		ordue.ACCOUNT,
		head.QUERYTIME,
		ordue.MONTH,
		month(date_format(head.QUERYTIME,'%Y-%m-%d')) - month(date_format(concat(ordue.month,'.01'),'%Y-%m-%d')) as gap_months,
		ordue.LASTMONTHS,
		ordue.AMOUNT,
		loanfo.GUARANTEETYPE,
		STATE,FINANCEORG
from ICRLATEST5YEAROVERDUEDETAIL ordue
-- 只取贷款的信息
inner join ICRLOANINFO loanfo on ordue.reportno=loanfo.reportno and ordue.ACCOUNT=loanfo.ACCOUNT 
left join ICRMESSAGEHEADER head on ordue.reportno=head.reportno
where month!='--'
""").createOrReplaceTempView("temp_loan_overdue")




spark.sql("""
select 
		reportno,
		ACCOUNT,
		QUERYTIME,
        MONTH,
		gap_months,
		LASTMONTHS,
		GUARANTEETYPE,
		STATE,
		FINANCEORG 
from temp_loan_2yearoverdue
union 
select 
		reportno,
		ACCOUNT,
		QUERYTIME,
		date_format(concat(month,'.01'),'%Y-%m'),gap_months,LASTMONTHS,
		GUARANTEETYPE,
		STATE,
		FINANCEORG 
from temp_loan_overdue
""").createOrReplaceTempView("temp_loan_5yearoverdue")


中间表已运行完成，spark sql 创建所需要的表

In [6]:
spark.sql("""
select 
		head.reportno,
		if(cdt.FIRSTLOANOPENMONTH='--' or cdt.FIRSTLOANOPENMONTH is null,-9999999,(month(date_format(head.QUERYTIME,'%Y-%m-%d'))-month(date_format(concat(FIRSTLOANOPENMONTH,'.01'),'%Y-%m-%d')) ) ) as first_loan_months,
		nvl(loan.loan_max_months,-9999999) as loan_max_months,
		nvl(loan.loan_min_months,-9999999) as loan_min_months,
		nvl(loan.loan_avg_months,-9999999) as loan_avg_months,
		nvl(lcdt.creditloan_max_months,-9999999) as creditloan_max_months,
		nvl(lcdt.creditloan_min_months,-9999999) as creditloan_min_months,
		nvl(lcdt.creditloan_avg_months,-9999999) as creditloan_avg_months,
		nvl(unst.unsettled_loan_max_months,-9999999) as unsettled_loan_max_months,
		nvl(unst.unsettled_loan_min_months,-9999999) as unsettled_loan_min_months,
		nvl(unst.unsettled_loan_avg_months,-9999999) as unsettled_loan_avg_months,
		nvl(uncdt.unsettled_creditloan_max_months,-9999999) as unsettled_creditloan_max_months,
		nvl(uncdt.unsettled_creditloan_min_months,-9999999) as unsettled_creditloan_min_months,
		nvl(uncdt.unsettled_creditloan_avg_months,-9999999) as unsettled_creditloan_avg_months,
        now() as create_time,now() as update_time,now() as etl_date
        
from ICRMESSAGEHEADER head 
left join ICRCREDITCUE cdt on head.reportno=cdt.reportno
left join 
(
	select 
		a.reportno,
		max(month(b.QUERYTIME)-month(OPENDATE)) as loan_max_months,
		min(month(b.QUERYTIME)-month(OPENDATE)) as loan_min_months,
		round(avg(month(b.QUERYTIME)-month(OPENDATE)),5) as loan_avg_months
	from ICRLOANINFO a
	left join ICRMESSAGEHEADER b on a.reportno=b.reportno
	group by a.reportno
)loan on head.reportno=loan.reportno
left join 
(
	select 
		a.reportno,
		max(month(b.QUERYTIME)-month(OPENDATE)) as creditloan_max_months,
		min(month(b.QUERYTIME)-month(OPENDATE)) as creditloan_min_months,
		round(avg(month(b.QUERYTIME)-month(OPENDATE)),5) as creditloan_avg_months
	from ICRLOANINFO a
	left join ICRMESSAGEHEADER b on a.reportno=b.reportno
	where GUARANTEETYPE='信用/免担保'
	group by a.reportno
)lcdt on head.reportno=lcdt.reportno
left join 
(
	select 
		a.reportno,
		max(month(b.QUERYTIME)-month(OPENDATE)) as unsettled_loan_max_months,
		min(month(b.QUERYTIME)-month(OPENDATE)) as unsettled_loan_min_months,
		round(avg(month(b.QUERYTIME)-month(OPENDATE)),5) as unsettled_loan_avg_months
	from ICRLOANINFO a
	left join ICRMESSAGEHEADER b on a.reportno=b.reportno
	where state in ('正常','呆账','逾期','转出')
	group by a.reportno
)unst on head.reportno=unst.reportno
left join 
(
	select 
		a.reportno,
		max(month(b.QUERYTIME)-month(OPENDATE)) as unsettled_creditloan_max_months,
		min(month(b.QUERYTIME)-month(OPENDATE)) as unsettled_creditloan_min_months,
		round(avg(month(b.QUERYTIME)-month(OPENDATE)),5) as unsettled_creditloan_avg_months
	from ICRLOANINFO a
	left join ICRMESSAGEHEADER b on a.reportno=b.reportno
	where state in ('正常','呆账','逾期','转出') and GUARANTEETYPE='信用/免担保'
	group by a.reportno
)uncdt on head.reportno=uncdt.reportno

""").createOrReplaceTempView("dw_loan_age")



In [7]:
table_name = "dw_loan_age"

spark.sql("drop table if EXISTS renhang_user_profile.%s "%table_name)
spark.sql("create table renhang_user_profile.%s as select * from %s"%(table_name,table_name))

DataFrame[]

In [1]:
spark.sql("""
select 
		head.reportno,
		if(cdt.FIRSTLOANOPENMONTH='--' or cdt.FIRSTLOANOPENMONTH is null,-9999999,timestampdiff(head.QUERYTIME, concat(FIRSTLOANOPENMONTH,'.01') )  ) as first_loan_months,
		nvl(loan.loan_max_months,-9999999) as loan_max_months,
		nvl(loan.loan_min_months,-9999999) as loan_min_months,
		nvl(loan.loan_avg_months,-9999999) as loan_avg_months,
		nvl(lcdt.creditloan_max_months,-9999999) as creditloan_max_months,
		nvl(lcdt.creditloan_min_months,-9999999) as creditloan_min_months,
		nvl(lcdt.creditloan_avg_months,-9999999) as creditloan_avg_months,
		nvl(unst.unsettled_loan_max_months,-9999999) as unsettled_loan_max_months,
		nvl(unst.unsettled_loan_min_months,-9999999) as unsettled_loan_min_months,
		nvl(unst.unsettled_loan_avg_months,-9999999) as unsettled_loan_avg_months,
		nvl(uncdt.unsettled_creditloan_max_months,-9999999) as unsettled_creditloan_max_months,
		nvl(uncdt.unsettled_creditloan_min_months,-9999999) as unsettled_creditloan_min_months,
		nvl(uncdt.unsettled_creditloan_avg_months,-9999999) as unsettled_creditloan_avg_months,
        now() as create_time,now() as update_time,now() as etl_date

from ICRMESSAGEHEADER head 
left join ICRCREDITCUE cdt on head.reportno=cdt.reportno
left join 
(
	select 
		a.reportno,
		max(cast(timestampdiff(b.QUERYTIME,OPENDATE) as int)) as loan_max_months,
		min(cast(timestampdiff(b.QUERYTIME,OPENDATE) as int)) as loan_min_months,
		round(avg(cast(timestampdiff(b.QUERYTIME,OPENDATE) as int)),5) as loan_avg_months
	from ICRLOANINFO a
	left join ICRMESSAGEHEADER b on a.reportno=b.reportno
	group by a.reportno
)loan on head.reportno=loan.reportno
left join 
(
	select 
		a.reportno,
		max(cast(timestampdiff(b.QUERYTIME,OPENDATE) as int)) as creditloan_max_months,
		min(cast(timestampdiff(b.QUERYTIME,OPENDATE) as int)) as creditloan_min_months,
		round(avg(cast(timestampdiff(b.QUERYTIME,OPENDATE) as int)),5) as creditloan_avg_months
	from ICRLOANINFO a
	left join ICRMESSAGEHEADER b on a.reportno=b.reportno
	where GUARANTEETYPE='信用/免担保'
	group by a.reportno
)lcdt on head.reportno=lcdt.reportno
left join 
(
	select 
		a.reportno,
		max(cast(timestampdiff(b.QUERYTIME,OPENDATE) as int)) as unsettled_loan_max_months,
		min(cast(timestampdiff(b.QUERYTIME,OPENDATE) as int)) as unsettled_loan_min_months,
		round(avg(cast(timestampdiff(b.QUERYTIME,OPENDATE) as int)),5) as unsettled_loan_avg_months
	from ICRLOANINFO a
	left join ICRMESSAGEHEADER b on a.reportno=b.reportno
	where state in ('正常','呆账','逾期','转出')
	group by a.reportno
)unst on head.reportno=unst.reportno
left join 
(
	select 
		a.reportno,
		max(cast(timestampdiff(b.QUERYTIME,OPENDATE) as int)) as unsettled_creditloan_max_months,
		min(cast(timestampdiff(b.QUERYTIME,OPENDATE) as int)) as unsettled_creditloan_min_months,
		round(avg(cast(timestampdiff(b.QUERYTIME,OPENDATE) as int)),5) as unsettled_creditloan_avg_months
	from ICRLOANINFO a
	left join ICRMESSAGEHEADER b on a.reportno=b.reportno
	where state in ('正常','呆账','逾期','转出') and GUARANTEETYPE='信用/免担保'
	group by a.reportno
)uncdt on head.reportno=uncdt.reportno

""").createOrReplaceTempView("dw_loan_age")



AnalysisException: "Undefined function: 'timestampdiff'. This function is neither a registered temporary function nor a permanent function registered in the database 'default'.; line 4 pos 76"